<a href="https://colab.research.google.com/github/jimxs74/adsl/blob/main/Experimen_Present_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentinel-1 and Sentinel-2 data fusion through Deep Learning**
![](imgs/fusion_paradigms.png)

**To re-train the models set the *TRAIN* flaf to *True*.**

**If *TRAIN = False* pre-trained weights will be loaded.**

Ambil dataset

Reference : https://github.com/wkentaro/gdown

source code : https://github.com/Sebbyraft/S1-S2-DataFusion

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download dataset dengan jumlah file cukup banyak dan besar

In [ ]:
# Tugas 1 : Jumlah dataset awal
# !unzip "/content/drive/MyDrive/400. COM_SCIE/465. Analisis data spasial lanjut/adsl research/S1-S2-DataFusion/dataset_zip.zip" > /dev/null

# Jumlah dataset awal diptimasi 50 pic /class
 !unzip "/content/drive/MyDrive/400. COM_SCIE/465. Analisis data spasial lanjut/adsl research/S1-S2-DataFusion/dataset_zip.zip" > /dev/null

In [24]:
# File Upload
import gdown
url_classifier = "https://drive.google.com/file/d/1BY_YU4GvG8HFMfXgldeVx0aDTch0lhv5/view?usp=sharing"
url_datasethandler = "https://drive.google.com/file/d/1BtSUZ6sZJze1uO-lMZ07jp7fHTP4V7Jz/view?usp=sharing"

output1 = "CNN_Classifier.py"
output2 = "DatasetHandler.py"

#gdown.download(url=url_classifier, output=output1, quiet=False, fuzzy=True)
#gdown.download(url=url_datasethandler, output=output2, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1BY_YU4GvG8HFMfXgldeVx0aDTch0lhv5
To: /content/CNN_Classifier.py
100%|██████████| 2.79k/2.79k [00:00<00:00, 5.09MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BtSUZ6sZJze1uO-lMZ07jp7fHTP4V7Jz
To: /content/DatasetHandler.py
100%|██████████| 9.93k/9.93k [00:00<00:00, 10.2MB/s]


'DatasetHandler.py'

In [25]:
TRAIN = True

**Settings**

In [26]:
EPOCHS = 10  # previous 30
BATCH_SIZE = 16 # perlu di modif tidak?

Install Library yang belum ada di Collabs

In [7]:
!pip install rasterio

     |████████████████████████████████| 19.3 MB 287 kB/s 


In [27]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

# Load dataset

The dataset is handeled by the class *DatasetHandler*.

In [28]:
!python DatasetHandler.py
!python CNN_Classifier.py

In [29]:
from DatasetHandler import DatasetHandler
training_handler = DatasetHandler('dataset/training')
validation_handler = DatasetHandler('dataset/validation')

In [30]:
print('Dataset classes')
for c in training_handler.classes: print('\t', c) #method .classes untuk ??

print('Dataset dimension')
print('\t', len(training_handler.s2_paths), 'training samples belonging to ', len(training_handler.classes), 'classes')
print('\t', len(validation_handler.s2_paths), 'validation samples belonging to ', len(validation_handler.classes), 'classes')

Dataset classes
	 dataset/training/sentinel2/city
	 dataset/training/sentinel2/coastline
	 dataset/training/sentinel2/lake
	 dataset/training/sentinel2/river
	 dataset/training/sentinel2/vegetation
Dataset dimension
	 450 training samples belonging to  5 classes
	 50 validation samples belonging to  5 classes


In [31]:
classes = []
for c in training_handler.classes:
    classes.append(c.split('/')[-1])

# Sentinel-2 classifier

![](imgs/sen2.png)

In [32]:
from CNN_Classifier import CNN_Classifier
s2classifier = CNN_Classifier((64,64, 12), 5)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [34]:
if TRAIN == True:
    batch_size = BATCH_SIZE
    training_loader = training_handler.s2_data_loader(batch_size, (64,64,12))
    validation_loader = validation_handler.s2_data_loader(batch_size, (64,64,12))
    training_steps = 4*len(training_handler.s2_paths)
    validation_steps = 4*len(validation_handler.s2_paths)
    epochs = EPOCHS
    s2classifier.train_model(epochs, batch_size, training_loader, validation_loader, training_steps, validation_steps)
    s2classifier.model.save('weights/S2-classifier.h5')
else:
    s2classifier.model = load_model('weights/S2-classifier.h5')

Epoch 1/10
1800/1800 [==============================] - 2865s 2s/step - loss: 0.3364 - accuracy: 0.6419 - val_loss: 0.3009 - val_accuracy: 0.7428
Epoch 2/10
1800/1800 [==============================] - 2886s 2s/step - loss: 0.2269 - accuracy: 0.7914 - val_loss: 0.3309 - val_accuracy: 0.6841
Epoch 3/10
1800/1800 [==============================] - 2849s 2s/step - loss: 0.1978 - accuracy: 0.8258 - val_loss: 0.3949 - val_accuracy: 0.7138
Epoch 4/10
 634/1800 [=========>....................] - ETA: 27:44 - loss: 0.1800 - accuracy: 0.8377

KeyboardInterrupt: ignored

# Sentinel-1 Classifier 

![](imgs/sen1.png)

In [ ]:
from CNN_Classifier import CNN_Classifier
s1classifier = CNN_Classifier((64,64, 2), 5)

In [ ]:
if TRAIN == True:
    training_handler = DatasetHandler('dataset/training')
    validation_handler = DatasetHandler('dataset/validation')
    batch_size = BATCH_SIZE
    training_loader = training_handler.s1_data_loader(batch_size, (64,64,2))
    validation_loader = validation_handler.s1_data_loader(batch_size, (64,64,2))
    training_steps = 4*len(training_handler.s1_paths)
    validation_steps = 4*len(validation_handler.s1_paths)
    epochs = EPOCHS
    s1classifier.train_model(epochs, batch_size, training_loader, validation_loader, training_steps, validation_steps)
    s1classifier.model.save('weights/S1-classifier.h5')
else:
    s1classifier.model = load_model('weights/S1-classifier.h5')

# Late Fusion

![](imgs/late-fusion.png)

In [ ]:
s2classifier = CNN_Classifier((64,64, 12), 5)
s1classifier = CNN_Classifier((64,64, 2), 5)
s2classifier.model = load_model('weights/S2-classifier.h5')
s1classifier.model = load_model('weights/S1-classifier.h5')

In [ ]:
validation_loader = validation_handler.s2_s1_data_loader(10*len(validation_handler.s1_paths), (64,64,12), (64,64,2))
s2_s1, g_truth = next(iter(validation_loader))
s2_pre = s2classifier.model.predict(s2_s1[0])
s1_pre = s1classifier.model.predict(s2_s1[1])

## Mean Late Fusion

In [ ]:
late_sum = []
for i in range(s2_pre.shape[0]):
        late_sum.append(np.argmax((s1_pre[i]+s2_pre[i])))

## Weighted Late Fusion

In [ ]:
late_weight = []
w1 = np.array([0, 1, 1, 1, 0])
w2 = 1 - w1

for i in range(s2_pre.shape[0]):
        late_weight.append(np.argmax((w1*s1_pre[i]+w2*s2_pre[i])))

# Results